# PyMetaSeem
CAMISIM es un `Programa` para simular datos metagenomicos a partir de genomas, a grandes rasgos este programa toma uno o varios archivos de genomas, y por un proceso de corte aleatorio de secuencias crea un archivo de datos metagenomicos.

A continuación se muestran los pasos para hacer una recreaccion de este algoritmo, y explicando cada una de las funciones creadas.

In [1]:
import os
import math
import random
import numpy as np

from pathlib import Path
from random import seed
from random import randint

La primera funcion se encarga de leer uno o varios genomas en archivos, los cuales se convertiran en un diccionario para poder trabajar con ellos en `Python`, en donde las `llaves`son los identificadores de los genoma y los `valores` la secuencia de nucleotidos.

In [2]:
# funcion para lectura de archivos
# ingresa un archivo multifasta y arroja una lista con cada secuencia (contig)
dicc_contigs = {}
def lectura_genoma(File):
    
    lista = []
    longitudes = []
    with open(File,'r') as f:
        lines=f.read() # lectura de cada linea del archivo
        lines=lines.split('>') # identificador de '>'
        lines=['>'+ x for x in lines[1:]] # lista con cada elemento que comienza con '>'
        
        for x in lines:
            x1 = x.replace(">","@")
            x2 = x1.replace("\n",",",1) # el primer '\n' se reemplazapor una coma
            x3 = x2.replace("\n","") # los siguientes se quitan
            lista.append(x3) # lista con un contig en cada elemento y su identificador
    
        # convertir la lista en un diccionario        
        for x in lista:
            x = x.split(',')
            dicc_contigs[x[0]] = x[1] 
        return(dicc_contigs)    

In [4]:
dicc_contigs = lectura_genoma('/home/csilva/GIT/Tesis_Maestria/Data/all-c-genomes(cortados)/GenomaPrueba.fna')

In [5]:
def cuenta_contigs(dicc_contigs):
    long_total = 0
    dicc_longitudes = {}
    num_contigs = len(dicc_contigs) # cuenta cuantos contigs contiene cada genoma (cuantas llaves)
    for key in dicc_contigs:
        longitud = len(dicc_contigs[key])# calcula la longitud de cada contig (la longitud de las claves, para cada llave)
        long_total +=  longitud # va sumando todas las longitudes de los contigs
        dicc_longitudes[key] = longitud # diccionario de longitudes 
        # key es el identificador del contig 
        # longitud -> es la longitud de cada contig
        # long_total -> lontitud total del genoma
    return (dicc_longitudes,num_contigs,long_total)   
        
        
        
        
### SE TOMA EL MINIMO DE CANTIDAD DE CONTIGS (se debe comparar por cada genoma)
### TOMAR EL NUMERO MINIMO DE CONTIGS CON MAYOR LONGITUD

In [6]:
dicc_longitudes,num_contigs,long_total = cuenta_contigs(dicc_contigs)

In [21]:
dicc_longitudes,num_contigs,long_total

({'@NZ_CP048049.1': 160, '@NZ_CP021034.1': 171, '@NZ_CP086349.1': 85}, 3, 416)

In [7]:
# funcion para calcular la proporcion de los contigs
def proporcion_contigs(dicc_contigs,dicc_longitudes,long_total):
    dicc_proporciones = {}
    for key in dicc_longitudes:
        P = dicc_longitudes[key]/long_total
        dicc_proporciones[key] = P
    return dicc_proporciones
# numero de reads por contig se calcula en la funcion reads 

In [8]:
dicc_proporciones = proporcion_contigs(dicc_contigs,dicc_longitudes,long_total)

In [ ]:
dicc_proporciones

Esta funcion se encarga de cortar una secuencia de nucleotidos dada, a partir de una pocision y una longitud dada.

In [9]:
# funcion para cortar reads, dado una pocision de inicio y una longitud de corte
def cutout(contig,i,n_length): #fordward    (resive contig y entrega reads)
    read = contig[i:i+n_length]
    return(read) 

In [ ]:
cutout(dicc_contigs['@NZ_CP048049.1'],5,10)

### Reads reverse

In [10]:
def cutout2(contig,i,n_length,inserto): #backward 
    # inserto = 400 (tamaño promedio del inserto), depende del secuenciador y lo da el usuario
    # inserto -> tamaño verdadero de la molecula de DNA que esta en el secuanciador
    j = i + inserto
    # j = i+inserto (pocision de inicio de reversa)
    # n_length = 150 (tamaño del read "cortado")
    read_cortado = contig[j-n_length:j]
    #i_read = contig[j-n_length:inserto]  #(quiero comenzar en la pocision j e ir hacia atras)
    return read_cortado

In [ ]:
cutout('ACTTGTAACTTGTAACTTGTAACTTGTAACTTGTATTGTATTGTAACTTG',5,10)

In [ ]:
cutout2('ACTTGTAACTTGTAACTTGTAACTTGTAACTTGTATTGTATTGTAACTTG',5,10,30)   # una funcion de solo corte y otra de solo reverse

In [ ]:
len('ACTTGTAACTTGTAACTTGTAACTTGTAACTTGTATTGTATTGTAACTTG')

In [11]:
# funcion que calcula el reverso de un Read 
def reverso(read_cortado):
    temp_list = list(read_cortado)
    temp_list.reverse()
    return ''.join(temp_list)

In [ ]:
reverso('GTAACTTGTA')  

In [79]:
def complementario(read_cortadoR):
    comp = []
    for i in range(len(read_cortadoR)):
        if read_cortadoR[i] == "A":
            comp.append("T")
        elif read_cortadoR[i] == "T":
            comp.append("A")
        elif read_cortadoR[i] == "G":
            comp.append("C")
        elif read_cortadoR[i] == "C":
            comp.append("G") 
            
    return "".join(comp)

In [76]:
complementario('ATGTTCAATG') 

'TACAATTAC'

In [13]:
def reverso_complementario(contig,i,n_length,inserto):
    read_rev_com = complementario(reverso(cutout2(contig,i,n_length,inserto))) 
    return (read_rev_com) # entrega cada read reverso complementario

In [ ]:
reverso_complementario('ACTTGTAACTTGTAACTTGTAACTTGTAACTTGTATTGTATTGTAACTTG',5,10,30)
# si se pone este read de ejemplo, el reverso complementario deberia quedar 'TAC'

Esta función se depende de la funcion anterior para cortar reads, y se complementa haciendo esto para cada genoma dentro de diccionario creado.

In [86]:
# funcion para cortar los reads aleatoriamente
# Al ingresar un conjunto de genomas, este nos arroja un metagenoma con varios reads tomados de los genomas iniciales
# para esto se le entrega el numero de reads deseados por cada genoma y la longitud.

def reads(dicc_contigs,dicc_longitudes,dicc_proporciones,n_length,num_reads_total):  
    
    dicc_reads = {}  # diccionario de reads 
    dicc_reads_reverso = {} # diccionario de reverso reads
    k = 0
    kk = 0
    inserto = 30 #400 (tamaño promedio del inserto), depende del secuenciador y lo da el usuario
     
    # LONGITUD Y NUM_READS DEPENDERA DE LAS LONGITUDES DE LOS CONTIG
    # LA LONGITUD 
    # LAS LONGITUDES DEPENDEN DE LA CALIDAD
    
    while k < num_reads_total: # número de reads total
              
        for key in dicc_contigs:
            #contig = random.choice(list(dicc_contigs.values())) # escoje un contig al azar
            num_reads_contig = round(num_reads_total* dicc_proporciones[key]) # numero de reads x proporcion #proporcion de reads por contig
            contig = dicc_contigs[key]
            # la proporcion viene del diccionario proporciones de la funcion cuenta contigs 
            for kk in range(num_reads_contig): 
                newkey = key + '_' + str(kk) # identificador de contig, con el numero de read
                i = randint(1,(len(contig)-n_length-inserto-1)) # toma una pocision de inicio al azar se debe tener en cuenta el inserto)
                dicc_reads[newkey] = cutout(contig,i,n_length) # para el contig dado anteriormente, se corta desde la pocision i de lonjitud ln
                dicc_reads_reverso[newkey] = reverso_complementario(contig,i,n_length,inserto)# reverso complementario
                #dicc_reads_reverso[newkey] = cutout2(contig,i,n_length,inserto)
                #dicc_reads_reverso[newkey] = reverso(cutout2(contig,i,n_length,inserto))
                #dicc_reads_reverso[newkey] = complementario(reverso(cutout2(contig,i,n_length,inserto))) ##AQUI ESTA EL PROBLEMA
                kk += 1
            k += num_reads_contig # funcion para cortar los reads aleatoriamente
        # se debe calcular cuantas veces va a escoger cada contig,dependiendo del tamaño del contig inicial      

    return(dicc_reads,dicc_reads_reverso)

In [87]:
dicc_reads,dicc_reads_reverso = reads(dicc_contigs,dicc_longitudes,dicc_proporciones,5,10)

In [88]:
dicc_reads_reverso

{'@NZ_CP048049.1_0': 'TGGGC',
 '@NZ_CP048049.1_1': 'CAGCA',
 '@NZ_CP048049.1_2': 'GGTGC',
 '@NZ_CP048049.1_3': 'TGACG',
 '@NZ_CP021034.1_0': 'AGATC',
 '@NZ_CP021034.1_1': 'GTGCA',
 '@NZ_CP021034.1_2': 'TGAAG',
 '@NZ_CP021034.1_3': 'TCGTC',
 '@NZ_CP086349.1_0': 'GTCGT',
 '@NZ_CP086349.1_1': 'CCGCG'}

In [89]:
dicc_reads

{'@NZ_CP048049.1_0': 'TGACC',
 '@NZ_CP048049.1_1': 'CCGAC',
 '@NZ_CP048049.1_2': 'GTCGA',
 '@NZ_CP048049.1_3': 'TCATC',
 '@NZ_CP021034.1_0': 'CCGAC',
 '@NZ_CP021034.1_1': 'GCGGA',
 '@NZ_CP021034.1_2': 'ACCGG',
 '@NZ_CP021034.1_3': 'GAAGG',
 '@NZ_CP086349.1_0': 'AAGGT',
 '@NZ_CP086349.1_1': 'GGCAG'}

In [ ]:
# funcion para simular la calidad

La ultima función toma el diccionario de reads creado por la anterior funcion, y lo guarda en un archivo `.fastq` , creando una calidad para cada read.

In [ ]:
# funcion de creacion archivo .fastq
    
def crea_fastq(dicc_reads,dicc_reads_reverso,n_length,file_name,file_name_reverse):
    file = open(file_name,'wt')
    for key in dicc_reads: 
        file.write(str(key))
        file.write(str('\n'))
        file.write(str(dicc_reads[key]))
        file.write(str('\n'))
        file.write(str('+'))
        file.write(str('\n'))
        file.write(str('A'*n_length)) #calidad
        file.write(str('\n'))     
        
        
    # agregar el reverso complementario  
    file = open(file_name_reverse,'wt')
    for key in dicc_reads_reverse: 
        file.write(str(key))
        file.write(str('\n'))
        file.write(str(dicc_reads_reverso[key]))
        file.write(str('\n'))
        file.write(str('+'))
        file.write(str('\n'))
        file.write(str('A'*n_length)) #calidad
        file.write(str('\n'))  
        
        
    file.close()
    

In [ ]:
crea_fastq(dicc_reads,dicc_reads_reverso,10,"prueba2.fastq")

#### Clase

A continuación se quiere agrupar todo dentro de una clase y poder realizar el proceso para varios archivos al mismo tiempo. Empezando con un solo archivo como prueba de la clase.

Esta clase se compone de cuatro funciones, las cuales:
 * Lee uno o varios archivos `.fasta` los cuales contienen genomas con uno o varios contigs.
 * Corta un read (secuencia genomica) de un tamaño dado y desde una posición aleatoria dada.
 * Crea un diccionario de reads metagenomicos de varios metagenomas dados.
 * Crea un archivo `.fastq` en donde guarda los datos metagenomicos añadiendo le la calidad.

In [ ]:
class CAMISIM_CAMI:
    def __init__(self, File, longitud ,num_reads):
        self.File = File
        self.longitud = longitud
        self.dic = {}
        self.dicReads = {}
        
        #longitud   (ejem = 150 por defauld o agusto del usuario)
        #numero de reads
        
    # funcion para lectura de archivos
    # ingresa un archivo multifasta y arroja una lista con cada secuencia (contig)
    def lectura_genoma(self):
        lista = []
        #with open(File,'r') as f: # con esta funcion podemos abrir un archivo en especifico con ruta completa
        with open(ruta + '/' + self.File,'r') as f:  # con esta funcion podemos leer los nombres de los archivos de la lista 'genomes' y le agrega la ruta para la lectura 
            lines=f.read() # lectura decada linea del archivo
            lines=lines.split('>') # identificador de '>'
            lines=['>'+ x for x in lines[1:]] # lista con cada elemento que comienza con '>'
            for x in lines:
                x1 = x.replace(">","@")
                x2 = x1.replace("\n",",",1) # el primer '\n' se reemplazapor una coma
                x3 = x2.replace("\n","") # los siguientes se quitan
                
                #reemplazar '>' con '@'
                
                lista.append(x3) # lista con un contig en cada elemento y su identificador
            # convertir la lista en un diccionario        
            for x in lista:
                x = x.split(',')
                self.dic[x[0]] = x[1] 
        return(self.dic)  
    
    # funcion para cortar reads, dado una pocision de inicio y una longitud de corte
    def cutout(self,read,i): #fordward
        cropped_read = read[i:i+n_length]
        return(cropped_read) 
    
    
    
    # funcion para cortar reads desde el final, dado una pocision de inicio, una longitud de corte y el tamaño del inserto 
    def cutout2(self,read,i,inserto): #backward 
        # inserto = 400 (tamaño promedio del inserto)
        j = i + inserto
        # j = i+inserto (pocision de inicio de reversa)
        # n_length = 150 (tamaño del read "cortado")
        cropped2_read = read[j-n_length:inserto]  #(quiero comenzar en la pocision j e ir hacia atras)
        return cropped2_read
    
    # crea el reverso del read ya cortado
    def reverso(self,read_cortado):
        temp_list = list(read_cortado)
        temp_list.reverse()
        return ''.join(temp_list)
    
    # crea el reverso complementario del read cortado 
    def reverso_complementario(self,read,i,inserto):
        result = complementario(reverso(cutout2(read,i,n_length,inserto)))
        return (result)



    
    # funcion para cortar los reads aleatoriamente
    # Al ingresar un conjunto de genomas, este nos arroja un metagenoma con varios reads tomados de los genomas iniciales
    # para esto se le entrega el numero de reads deseados por cada genoma y la longitud.
    def reads(self,num_reads):  # num_reads
        contig = 0
        k = 0
        while k < num_reads:
            for key in self.dic:#range(len(diccionario.keys())):
                newkey = key + '_' + str(k) 
                contig = random.choice(list(self.dic.values())) # escoje un contig al azar 
                i = randint(1,(len(contig)-self.longitud)) # toma una pocision de inicio al azar
                self.dicReads[newkey] = cutout(contig,i,self.longitud) # para el contig dado anteriormente, se corta desde la pocision i de lonjitud ln
                # se debe calcular cuantas veces va a escoger cada contig,dependiendo del tamaño del contig inicial      
            k += 1
        return(self.dicReads)
    
    # funcion de creacion archivo .fastq, tomandoel diccionario de reads cortados y la calidad de los reads
    def crea_fastq(diccionario,longitud,file_name):
        file = open(file_name,'wt')
        for key in diccionario: 
            file.write(str(key))
            file.write(str('\n'))
            file.write(str(diccionario[key]))
            file.write(str('\n'))
            file.write(str('+'))
            file.write(str('\n'))
            file.write(str('A'*longitud)) #calidad
            file.write(str('\n'))     
            
            
            
            
        file.close()
    

In [ ]:
A=CAMISIM_CAMI("Clavibacter_michiganensis_subsp_capsici_1101.fna-cortado.fna",5,10)

In [ ]:
A.reads(4)

In [ ]:
dicreads = CAMISIM_CAMI.reads(dic,10) #???

In [ ]:
CAMISIM_CAMI.crea_fastq(dicreads,10,"prueba2.fastq")

Y por ultimo realizandole unas pocas modificaciones a la clase, para poder realizar el proceso para varios archivos de genomas.

y aplicando lo visto en clase sobre lectura de arhivos, e iteracion sobre cada uno de ellos.

In [ ]:
# 

In [ ]:
# Funcion para la lectura de los nombres de los todos los archivos de genomas
# toma la ruta de la carpeta donde estan los genomas, y crea un listado de los nombres de los archivos
# para poder realizar la lectura de los mismos.
ruta = '/home/csilva/GIT/Tesis_Maestria/Data/all-c-genomes(cortados)'
contenido = os.listdir(ruta)
genomes = []
for i in contenido:
    if os.path.isfile(os.path.join(ruta,i)) and i.endswith('.fna'):
        genomes.append(i)
n = len(genomes) # número de genomas (número de archivos)
diccionario_reads_metagenoma = {}
    

In [ ]:
#genomes

In [ ]:
# Con esta funcion queremos llamar a las funciones de la clase,para todos los archivos en la lista 'genomes'

def todo(genomes,longitud,num_reads,file_name):

    for i,x in enumerate(genomes):
    
        CAMISIM_CAMI.lectura_genoma(x)                        #
        dicreads = CAMISIM_CAMI.reads(dic,longitud,num_reads) #funcion construcctora
        fasta = CAMISIM_CAMI.crea_fastq(dicreads,longitud,file_name)
#print(dic)
    
    

In [ ]:
todo(genomes,150,5,'prueba_clase.fastq')

Para Shaday:
* 9 archivos  

1. (3 de un solo genoma)
    * 1.1
        1. Clavibacter_michiganensis_subsp_capsici_PF008.fna-cortado.fna

    * 1.2
        1. Clavibacter_michiganensis_subsp_nebraskensis_CIBA.fna-cortado.fna

    * 1.3
        1. Clavibacter_michiganensis_subsp_sepedonicus_CFIA-Cs3N.fna-cortado.fna
        
2. (3 de 2) 
    * 2.1
        1. Clavibacter_michiganensis_subsp_capsici_1106.fna-cortado.fna
        2. Clavibacter_michiganensis_subsp_nebraskensis_7580.fna-cortado.fna
        
    * 2.2
        1. Clavibacter_michiganensis_subsp_insidiosus_LMG_3663.fna-cortado.fna
        2. Clavibacter_michiganensis_subsp_tessellarius_ATCC_33566.fna-cortado.fna
        
    * 2.3
        1. Clavibacter_michiganensis_subsp_sepedonicus_ATCC33113.fna-cortado.fna
        2. Clavibacter_michiganensis_subsp_tessellarius_ATCC_33566.fna-cortado

3. (3 de 3)
    * 3.1
        1. Clavibacter_michiganensis_subsp_capsici_1101.fna-cortado.fna
        2. Clavibacter_michiganensis_subsp_nebraskensis_CFBP_7577.fna-cortado.fna
        3. Clavibacter_michiganensis_subsp_sepedonicus_CFIA-CsR14.fna-cortado.fna  
        
    * 3.2
        1. Clavibacter_michiganensis_subsp_insidiosus_CFBP_6488.fna-cortado.fna
        2. Clavibacter_michiganensis_subsp_sepedonicus_CFIA-CsR14.fna-cortado.fna   
        3. Clavibacter_michiganensis_subsp_tessellarius_ATCC33113.fna-cortado.fna
        
    * 3.3
        1. Clavibacter_michiganensis_subsp_insidiosus_CFBP_2404.fna-cortado.fna
        2. Clavibacter_michiganensis_subsp_tessellarius_ATCC_33566.fna-cortado.fna
        3. Clavibacter_michiganensis_subsp_sepedonicus_CFIA-CsR14.fna-cortado.fna  


In [ ]:
# 3 listas con 1 genoma
genomes11 = ['Clavibacter_michiganensis_subsp_capsici_PF008.fna-cortado.fna']
genomes12 = ['Clavibacter_michiganensis_subsp_nebraskensis_CIBA.fna-cortado.fna']
genomes13 = ['Clavibacter_michiganensis_subsp_sepedonicus_CFIA-Cs3N.fna-cortado.fna']
# 3 listas con 2 genomas
genomes21 = ['Clavibacter_michiganensis_subsp_capsici_1106.fna-cortado.fna','Clavibacter_michiganensis_subsp_nebraskensis_7580.fna-cortado.fna']
genomes22 = ['Clavibacter_michiganensis_subsp_insidiosus_LMG_3663.fna-cortado.fna','Clavibacter_michiganensis_subsp_tessellarius_ATCC_33566.fna-cortado.fna']
genomes23 = ['Clavibacter_michiganensis_subsp_sepedonicus_ATCC33113.fna-cortado.fna','Clavibacter_michiganensis_subsp_tessellarius_ATCC_33566.fna-cortado']
# 3 listas con 3 genomas
genomes31 = ['Clavibacter_michiganensis_subsp_capsici_1101.fna-cortado.fna','Clavibacter_michiganensis_subsp_nebraskensis_CFBP_7577.fna-cortado.fna','Clavibacter_michiganensis_subsp_sepedonicus_CFIA-CsR14.fna-cortado.fna']
genomes32 = ['Clavibacter_michiganensis_subsp_insidiosus_CFBP_6488.fna-cortado.fna','Clavibacter_michiganensis_subsp_sepedonicus_CFIA-CsR14.fna-cortado.fna','Clavibacter_michiganensis_subsp_tessellarius_ATCC33113.fna-cortado.fna']
genomes33 = ['Clavibacter_michiganensis_subsp_insidiosus_CFBP_2404.fna-cortado.fna','Clavibacter_michiganensis_subsp_tessellarius_ATCC_33566.fna-cortado.fna','Clavibacter_michiganensis_subsp_sepedonicus_CFIA-CsR14.fna-cortado.fna'] 

In [ ]:
# como calcular la calidada de las secuencias 